In [ ]:
'''
ATTENTION: 

Please follow the following instructions to test out the prototype!

1. Install PostGreSQL package from: (13)
    https://www.enterprisedb.com/downloads/postgres-postgresql-downloads
    
2. Install pgAdmin webapp at: (4.25)
    https://www.pgadmin.org/
    
3. Create a localhost database with the following commands in SQL Shell (included with PostgreSQL):
    >>>Server [localhost]
    >>>Database [Postgres]
    >>>Port [5432]
    >>>Username [Postgres]
    >>>Password [User_Defined]
    
    *** REMEMBER TO CHANGE THE PARAMETERS IN THE <construct_pg_url> function in python accordingly ***
    
4. PLace the python codes, JSON files in a single folder 

5. Run the pythin in the following order: (or sequentially in the jupyter notebook)
    >>>table_init.py
    >>>PSQL_method.py
    >>>JSON_method.py
    
6.Sit back and enjoy! :0)

CHEERS M8
'''

In [ ]:
#initialize table in PostgreSQL database
#please just run this once, or else your DB is going to have a hard time loading

import dataset
import json

def construct_pg_url(postgres_user='postgres', postgres_password='as457d', postgres_host='localhost', postgres_port='5432', postgres_database='speckle_uno'):
    return ("postgresql://" + postgres_user + ":" + postgres_password + '@' + postgres_host + ':' + postgres_port + '/' + postgres_database)

with open ("RHX1FSNpM.json" ) as e:
    sID = "RHX1FSNpM"
    sData = json.load(e)
    
    db = dataset.connect(construct_pg_url())
    table1 = db['table_uno']
    table1.upsert(dict(stream_id = sID, stream_data = sData), ['1'])
    
e.close()

In [ ]:
#index Postgres objects
import dataset
import json

def construct_pg_url(postgres_user='postgres', postgres_password='as457d', postgres_host='localhost', postgres_port='5432', postgres_database='speckle_uno'):
    return ("postgresql://" + postgres_user + ":" + postgres_password + '@' + postgres_host + ':' + postgres_port + '/' + postgres_database)

db = dataset.connect(construct_pg_url())
db.query('CREATE INDEX idx_type ON table_uno ((stream_data->\'resources\'->\'type\'));')
db.query('CREATE INDEX idx_mat ON table_uno ((stream_data->\'resources\'->\'materialRef\'));')


In [7]:
%%time

import json
import math
from collections import OrderedDict

#filters
def filter_element(data, element_type):
    resources = data['resources']
    filter = [d for d in resources if any([(d['type'] == e) for e in element_type])]
    return filter
def filter_properties(data):
    resources = data['resources']
    filter = [d for d in resources if 'materialRef' in d]
    return filter
    
    
#component functions for volume and length modules
def find_propertyRef(lista):
    return lista['properties']['structural']['propertyRef']

def find_topology_list(lista):
    return lista['value']

def find_element_elevation(lista):
    topology_array = lista['value']
    if (topology_array[2]>topology_array[-1]): return topology_array[2]
    else: return topology_array[-1]

def area_2_list(a_list):
    """Returns the area when given a list of x,y,z,x,y,z... coordinates"""
    if len(a_list) % 3 != 0:
        raise ValueError('List does not contain multiples of three values')
    if a_list[:3] != a_list[-3:]:
        a_list += a_list[:3] # add first to end to close the perimeter
    
    x1, y1, _, *a_list = a_list
    area = 0.0
    while a_list:
        x2, y2, _, *a_list = a_list
        area += (x1 * y2 - x2 * y1) / 2.0
        x1, y1 = x2, y2
    return area

def gen_property_dict(data):
    property_dict = {}
    for entry in data:
        if (entry['type'] == "Structural1DProperty"):
            property_dict[entry['applicationId']] = area_2_list(entry['profile']['value'])
    return property_dict

def gen_volume_by_elev(input_dict, volume, elev):
    if input_dict.get(elev): input_dict[elev] = input_dict[elev] + volume
    else: input_dict[elev] = volume
    input_dict = OrderedDict(sorted(input_dict.items(), key=lambda c: c[0]))
    return input_dict

def printer(_label, input_dict, color, _type):
    list_keys = []
    list_values = []
    output_dict = {}
    for entry in input_dict:
        list_keys.append(entry)
        list_values.append(input_dict[entry])
    output_dict["label"] = _label
    output_dict["backgroundColor"] = color
    output_dict['labels'] = list_keys
    output_dict["data"] = list_values
    output_dict["type"] = _type
    return output_dict
    
#Modules
def length(topo):
    return math.sqrt((abs(topo[0] - topo[-3])**2) + (abs(topo[1] - topo[-2])**2) + (abs(topo[2] - topo[-1])**2))
    
def unit_volumn(lista, entry):
    index =  lista[entry]
    return float(index)

def element_volume(length, unit_volume):
     return (length * unit_volume)
    
    
#main stuff
def volume_converter (inputjson):
    with open( inputjson ) as f:
        speckle_objects = json.load(f)
        property_dict = gen_property_dict(filter_properties(speckle_objects)) 
        filter1D = filter_element(speckle_objects, ['Polyline/Structural1DElementPolyline', 'Line/Structural1DElement'])

        number_of_elements = len(filter1D)
        total_volume = 0 #function deprecated
        volume_elev_dict = {}
        
        for d in filter1D:
            element_v = element_volume(length(find_topology_list(d)), unit_volumn(property_dict, find_propertyRef(d)))
            total_volume += element_v
            volume_elev_dict = gen_volume_by_elev(volume_elev_dict, element_v, find_element_elevation(d))
        return(volume_elev_dict)

def labels_extracter (input_dict):
    input_dict.pop('labels')
    return input_dict

def elevation_scanner (lista):
    keys = list(set(sum([list(d.keys()) for d in lista],[])))
    keys.sort()
    return [{k:d.get(k,0) for k in keys} for d in lista]

def multiple_mod_conversion(streamID_list): #multiple model conversion
    color_list = ['#ff6666','#ff8c66','#ffb366','#ffd966','#ffff66','#d9ff66','#b3ff66','#8cff66','#66ff66','#66ff8c','#66ffb3','#66ffd9','#66ffff','#66d9ff','#66b3ff','#668cff','#6666ff','#8c66ff','#b366ff','#d966ff','#ff66ff','#ff66d9','#ff66b3','#ff668c','#ff6666']
    color_index = ((4 * i) % 25 for i in range(len(color_list)))
    stream_results_list = [volume_converter(ID) for ID in streamID_list]
    return [labels_extracter(printer(streamID_list[x], elevation_scanner(stream_results_list)[x], color_list[next(color_index)],'bar')) for x in range (len(streamID_list))]

def comparison_conversion(comparison_list): #comparision chart data generation
    output_list = []
    color_list = ['#ff6666','#ff8c66','#ffb366','#ffd966','#ffff66','#d9ff66','#b3ff66','#8cff66','#66ff66','#66ff8c','#66ffb3','#66ffd9','#66ffff','#66d9ff','#66b3ff','#668cff','#6666ff','#8c66ff','#b366ff','#d966ff','#ff66ff','#ff66d9','#ff66b3','#ff668c','#ff6666']
    color_index = ((4 * i) % 25 for i in range(len(color_list)))
    color_list[next(color_index)]
    comparison_results_list = [volume_converter(ID) for ID in comparison_list]
    comparison_list = [labels_extracter(printer(comparison_list[x], elevation_scanner(comparison_results_list)[x], 'none', 'bar')) for x in range (len(comparison_list))]
    data_list = [comparison_list['data'] for comparison_list in comparison_list]
    for record in range(len(data_list)-1):
        output_list.append({'label': 'comparison', 'data': [(data_list[record+1][x]-data_list[0][x]) for x in range(len(data_list[0]))], 'backgroundColor': color_list[next(color_index)], 'type': 'bar'})
    return output_list

results_dict_1 = {}
results_dict_2 = {}
streamID_list = ["RHX1FSNpM.json", "QSGAnXVdI.json", "taNRrjz3u.json"]

results_dict_1['datasets'] = multiple_mod_conversion(streamID_list)
results_dict_1['labels'] = [3.5, 7, 10.5]
results_dict_2['datasets'] = comparison_conversion(streamID_list)
results_dict_2['labels'] = [3.5, 7, 10.5]
    
with open( "out_data.json", 'w') as writefile:
    json.dump(results_dict_1, writefile)
    json.dump(results_dict_2, writefile)
    print(results_dict_1)
    print(results_dict_2)

 
 

{'datasets': [{'label': 'RHX1FSNpM.json', 'backgroundColor': '#ff6666', 'data': [58.90700000000001, 74.20800000000001, 1.5311852192367725], 'type': 'bar'}, {'label': 'QSGAnXVdI.json', 'backgroundColor': '#ffff66', 'data': [69.40700000000001, 60.024999999999984, 1.5311852192367719], 'type': 'bar'}, {'label': 'taNRrjz3u.json', 'backgroundColor': '#66ff66', 'data': [69.40700000000002, 63.707999999999984, 1.5311852192367719], 'type': 'bar'}], 'labels': [3.5, 7, 10.5]}
{'datasets': [{'label': 'comparison', 'data': [10.5, -14.183000000000028, -6.661338147750939e-16], 'backgroundColor': '#ffff66', 'type': 'bar'}, {'label': 'comparison', 'data': [10.500000000000014, -10.500000000000028, -6.661338147750939e-16], 'backgroundColor': '#66ff66', 'type': 'bar'}], 'labels': [3.5, 7, 10.5]}
Wall time: 954 ms


In [24]:
%%timeit

import dataset
import json
import math

def construct_pg_url(postgres_user='postgres', postgres_password='as457d', postgres_host='localhost', postgres_port='5432', postgres_database='speckle_uno'):
    return ("postgresql://" + postgres_user + ":" + postgres_password + '@' + postgres_host + ':' + postgres_port + '/' + postgres_database)

db = dataset.connect(construct_pg_url())
    
#filters
def filter_element(datab):
    element_list = []
    statement = 'SELECT * FROM (SELECT jsonb_array_elements(stream_data->\'resources\')AS resources FROM table_uno) AS main_table WHERE (resources->>\'type\' = \'Polyline/Structural1DElementPolyline\') OR (resources->>\'type\' = \'Line/Structural1DElement\');'
    for row in datab.query(statement):
        element_list.append(row)
    return element_list

def filter_property(datab):
    property_list = []
    statement = 'SELECT * FROM (SELECT jsonb_array_elements(stream_data->\'resources\')AS resources FROM table_uno) AS main_table WHERE resources->>\'materialRef\' IS NOT NULL;'
    for row in datab.query(statement):
        property_list.append(row)
    return property_list

#component functions for volume and length modules
def find_propertyRef(lista):
    return lista['resources']['properties']['structural']['propertyRef']

def find_topology_list(lista):
    return lista['resources']['value']

def find_element_elevation(lista):
    topology_array = lista['resources']['value']
    if (topology_array[2]>topology_array[5]): return topology_array[2]
    else: return topology_array[5]

def area_2_list(a_list):
    """Returns the area when given a list of x,y,z,x,y,z... coordinates"""
    if len(a_list) % 3 != 0:
        raise ValueError('List does not contain multiples of three values')
    if a_list[:3] != a_list[-3:]:
        a_list += a_list[:3] 
    
    x1, y1, _, *a_list = a_list
    area = 0.0
    while a_list:
        x2, y2, _, *a_list = a_list
        area += (x1 * y2 - x2 * y1) / 2.0
        x1, y1 = x2, y2
    return area

def gen_property_dict(data):
    property_dict = {}
    for entry in data:
        if (entry['resources']['type'] == "Structural1DProperty"):
            property_dict[entry['resources']['applicationId']]=area_2_list(entry['resources']['profile']['value'])
    return property_dict

def gen_volume_by_elev(input_dict, volume, elev):
    if input_dict.get(elev): input_dict[elev] = input_dict[elev] + volume
    else: input_dict[elev] = volume
    return input_dict

def printer(input_dict):
    list_keys = []
    list_values = []
    output_dict = {}
    for entry in input_dict:
        list_keys.append(entry)
        list_values.append(input_dict[entry])
    output_dict["list_keys"] = list_keys
    output_dict["list_values"] = list_values
    return output_dict

#Modules
def length(topo):
    return math.sqrt((abs(topo[0] - topo[-3])**2) + (abs(topo[1] - topo[-2])**2) + (abs(topo[2] - topo[-1])**2))
    
def unit_volumn(lista, entry):
    index =  lista[entry]
    return float(index)

def element_volume(length, unit_volume):
     return (length * unit_volume)
    
#main stuff
with open( "out_data.json", 'w') as writefile:
    property_dict = gen_property_dict(filter_property(db)) #reference
    filter1D = filter_element(db)

    number_of_elements = len(filter1D)
    total_volume = 0
    print("Number of Records: " + str(number_of_elements))

    volume_elev_dict = {}

    for d in filter1D:
        element_v = element_volume(length(find_topology_list(d)), unit_volumn(property_dict, find_propertyRef(d)))
        total_volume += element_v
        volume_elev_dict = gen_volume_by_elev(volume_elev_dict, element_v, find_element_elevation(d))

    print("total volume of structural 1D members: ",total_volume)
    print(volume_elev_dict)
    json.dump(printer(volume_elev_dict), writefile)


Number of Records: 142
total volume of structural 1D members:  134.64618521923686
{10.5: 1.5311852192367725, 7: 53.208, 3.5: 79.90700000000001}
Number of Records: 142
total volume of structural 1D members:  134.64618521923686
{10.5: 1.5311852192367725, 7: 53.208, 3.5: 79.90700000000001}
Number of Records: 142
total volume of structural 1D members:  134.64618521923686
{10.5: 1.5311852192367725, 7: 53.208, 3.5: 79.90700000000001}
Number of Records: 142
total volume of structural 1D members:  134.64618521923686
{10.5: 1.5311852192367725, 7: 53.208, 3.5: 79.90700000000001}
Number of Records: 142
total volume of structural 1D members:  134.64618521923686
{10.5: 1.5311852192367725, 7: 53.208, 3.5: 79.90700000000001}
Number of Records: 142
total volume of structural 1D members:  134.64618521923686
{10.5: 1.5311852192367725, 7: 53.208, 3.5: 79.90700000000001}
Number of Records: 142
total volume of structural 1D members:  134.64618521923686
{10.5: 1.5311852192367725, 7: 53.208, 3.5: 79.90700000